In [1]:
%matplotlib inline

import copy
import os
import pickle
from collections import defaultdict
import itertools

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.cluster import AgglomerativeClustering


from context import messi
from messi.data_processing import *
from messi.hme import hme
from messi.gridSearch import gridSearch

# Reproduction of MESSI results on MERFISH

Here we reproduce MESSI's results shown in our manuscript, using the excitatory neurons in the female naive animals of the MERFISH hypothalamus dataset as an example. The corresponding results are shown in Figure 3. 

In order to reproduce the results in the manuscript, the number of experts and other hyperparameters are specified according to our grid search results as shown in Table S3 in our manuscript.

The results included in this notebook were produced in Linux Mint 19 with Intel(R) Xeon(R) W-2123 CPU @ 3.60GHz and 16 GB memory.

## User-definied arguements

Same as the arguements for the command-line usage. See detailed information of each arguement in the docs. 

In [3]:
input_path = '../../input/merfish/'
output_path = '../output/'

data_type = 'merfish'
sex = 'Female'
behavior = "Naive" 
behavior_no_space = behavior.replace(" ", "_")
current_cell_type = 'Excitatory'
current_cell_type_no_space = current_cell_type.replace(" ", "_")

grid_search = False 
n_sets = 5  

n_classes_0 = 1
# n_classes_1 = 8 # we will specify number of experts later
n_epochs = 20  

preprocess = 'neighbor_cat'
top_k_response = None  
top_k_regulator = None
response_type = 'original'  # use raw values to fit the model

mode = "CV"
n_replicates = 1

## Read in and preprocess data

### Read in the ligad & receptor list and meta information for the dataset

The meta information can be obtained by running readyData.py. See the details in the docs. 

In [4]:
read_in_functions = {'merfish': [read_meta_merfish, read_merfish_data, get_idx_per_dataset_merfish],
                    'merfish_cell_line': [read_meta_merfish_cell_line, read_merfish_cell_line_data, get_idx_per_dataset_merfish_cell_line],
                    'starmap': [read_meta_starmap_combinatorial, read_starmap_combinatorial, get_idx_per_dataset_starmap_combinatorial]}

# set data reading functions corresponding to the data type
if data_type in ['merfish', 'merfish_cell_line', 'starmap']:
    read_meta = read_in_functions[data_type][0]
    read_data = read_in_functions[data_type][1]
    get_idx_per_dataset = read_in_functions[data_type][2]
else:
    raise NotImplementedError(f"Now only support processing 'merfish', 'merfish_cell_line' or 'starmap'")

# read in ligand and receptor lists
l_u, r_u = get_lr_pairs(input_path='../messi/input/')  # may need to change to the default value

# read in meta information about the dataset
meta_all, meta_all_columns, cell_types_dict, genes_list, genes_list_u, \
response_list_prior, regulator_list_prior = \
    read_meta(input_path, behavior_no_space, sex, l_u, r_u)  # TO BE MODIFIED: number of responses

# get all available animals/samples
all_animals = list(set(meta_all[:, meta_all_columns['Animal_ID']]))

Removed genes: {'Blank_1', 'Blank_5', 'Blank_3', 'Blank_2', 'Fos', 'Blank_4'}
Total number of cell types for merfish: 16


# Reproduce results by running outer CV

In order to reproduce the results in the manuscript, here we set the number of experts and the other hyperparameters the same values as we present in Table S3 in the manuscript. 

In [5]:
n_experts_types = {'Inhibitory': {1: 10, 2: 10, 3: 10, 4: 10}, 
                   'Excitatory': {1: 8, 2: 8, 3: 10, 4: 10},
                   'Astrocyte': {1: 4, 2: 4, 3: 3, 4: 3},
                   'OD Mature 2' : {1: 3, 2: 3, 3: 4, 4: 3},
                   'Endothelial 1': {1: 1, 2: 1, 3: 2, 4: 2},
                   'OD Immature 1': {1: 1, 2: 1, 3: 2, 4: 2},
                   'OD Mature 1': {1: 1, 2: 1, 3: 1, 4: 1},
                   'Microglia': {1: 1, 2: 1, 3: 1, 4: 1}}

soft_weights_types = {'Inhibitory': {1: True, 2: True, 3: True, 4: True}, 
                    'Excitatory': {1: True, 2: True, 3: True, 4: True}, 
                    'Astrocyte': {1: True, 2: True, 3: False, 4: False}, 
                    'OD Mature 2' : {1: True, 2: True, 3: True, 4: False}, 
                    'Endothelial 1': {1: True, 2: True, 3: False, 4: False}, 
                    'OD Immature 1': {1: True, 2: True, 3: True, 4: True}, 
                    'OD Mature 1': {1: True, 2: True, 3: True, 4: True}, 
                    'Microglia': {1: True, 2: True, 3: True, 4: True}}

In [6]:
import time
start_time = time.time()

In [7]:
for _z in range(len(all_animals)):
    if mode == 'train':
        # only run once
        if _z == 0:
            test_animal = ''
        else:
            break
    else:
        test_animal = all_animals[_z]
    
    # specify number of experts and other hyperparameters according to Table S3
    n_classes_1 = n_experts_types[current_cell_type][test_animal]
    soft_weights = soft_weights_types[current_cell_type][test_animal]
    partial_fit_expert = soft_weights
    
    condition = f"response_{top_k_response}_l1_{n_classes_0}_l2_{n_classes_1}"

    if grid_search:
        condition = f"response_{top_k_response}_l1_{n_classes_0}_l2_grid_search"
    else:
        condition = f"response_{top_k_response}_l1_{n_classes_0}_l2_{n_classes_1}"

    
    samples_test = np.array([test_animal])
    samples_train = np.array(list(set(all_animals) - {test_animal}))
    print(f"Test set is {samples_test}")
    print(f"Training set is {samples_train}")

    bregma = None
    # ------ read data ------
    idx_train, idx_test, idx_train_in_general, \
    idx_test_in_general, idx_train_in_dataset, \
    idx_test_in_dataset, meta_per_dataset_train, \
    meta_per_dataset_test = find_idx_for_train_test(samples_train, samples_test,
                                                    meta_all, meta_all_columns,
                                                    data_type, current_cell_type, get_idx_per_dataset,
                                                    return_in_general=False, bregma=bregma)

    # TBD: the current approach uses a lot memory;
    data_sets = []

    for animal_id, bregma in meta_per_dataset_train:
        hp, hp_cor, hp_genes = read_data(input_path, bregma, animal_id, genes_list, genes_list_u)

        if hp is not None:
            hp_columns = dict(zip(hp.columns, range(0, len(hp.columns))))
            hp_np = hp.to_numpy()
        else:
            hp_columns = None
            hp_np = None
        hp_cor_columns = dict(zip(hp_cor.columns, range(0, len(hp_cor.columns))))
        hp_genes_columns = dict(zip(hp_genes.columns, range(0, len(hp_genes.columns))))
        data_sets.append([hp_np, hp_columns, hp_cor.to_numpy(), hp_cor_columns,
                          hp_genes.to_numpy(), hp_genes_columns])
        del hp, hp_cor, hp_genes

    datasets_train = data_sets

    data_sets = []

    for animal_id, bregma in meta_per_dataset_test:
        hp, hp_cor, hp_genes = read_data(input_path, bregma, animal_id, genes_list, genes_list_u)

        if hp is not None:
            hp_columns = dict(zip(hp.columns, range(0, len(hp.columns))))
            hp_np = hp.to_numpy()
        else:
            hp_columns = None
            hp_np = None

        hp_cor_columns = dict(zip(hp_cor.columns, range(0, len(hp_cor.columns))))
        hp_genes_columns = dict(zip(hp_genes.columns, range(0, len(hp_genes.columns))))
        data_sets.append([hp_np, hp_columns, hp_cor.to_numpy(), hp_cor_columns,
                          hp_genes.to_numpy(), hp_genes_columns])
        del hp, hp_cor, hp_genes

    datasets_test = data_sets

    del data_sets

    # ------ pre-processing -------

    # construct neighborhood graph
    if data_type == 'merfish_RNA_seq':
        neighbors_train = None
        neighbors_test = None
    else:
        if data_type == 'merfish':
            dis_filter = 100
        else:
            dis_filter = 1e9

        neighbors_train = get_neighbors_datasets(datasets_train, "Del", k=10, dis_filter=dis_filter,
                                                 include_self=False)
        neighbors_test = get_neighbors_datasets(datasets_test, "Del", k=10, dis_filter=dis_filter,
                                                include_self=False)
    # set parameters for different feature types
    lig_n = {'name': 'regulators_neighbor', 'helper': preprocess_X_neighbor_per_cell,
             'feature_list_type': 'regulator_neighbor', 'per_cell': True, 'baseline': False,
             'standardize': True, 'log': True, 'poly': False}
    rec_s = {'name': 'regulators_self', 'helper': preprocess_X_self_per_cell,
             'feature_list_type': 'regulator_self', 'per_cell': True, 'baseline': False,
             'standardize': True, 'log': True, 'poly': False}
    lig_s = {'name': 'regulators_neighbor_self', 'helper': preprocess_X_self_per_cell,
             'feature_list_type': 'regulator_neighbor', 'per_cell': True, 'baseline': False,
             'standardize': True, 'log': True, 'poly': False}
    type_n = {'name': 'neighbor_type', 'helper': preprocess_X_neighbor_type_per_dataset,
              'feature_list_type': None, 'per_cell': False, 'baseline': False,
              'standardize': True, 'log': False, 'poly': False}
    base_s = {'name': 'baseline', 'helper': preprocess_X_baseline_per_dataset, 'feature_list_type': None,
              'per_cell': False, 'baseline': True, 'standardize': True, 'log': False, 'poly': False}

    if data_type == 'merfish_cell_line':
        feature_types = [lig_n, rec_s, base_s, lig_s]
    else:
        feature_types = [lig_n, rec_s, type_n, base_s, lig_s]

    # untransformed features
    X_trains, X_tests, regulator_list_neighbor, regulator_list_self = prepare_features(data_type, datasets_train,
                                                                                       datasets_test,
                                                                                       meta_per_dataset_train,
                                                                                       meta_per_dataset_test,
                                                                                       idx_train, idx_test,
                                                                                       idx_train_in_dataset,
                                                                                       idx_test_in_dataset,
                                                                                       neighbors_train,
                                                                                       neighbors_test,
                                                                                       feature_types,
                                                                                       regulator_list_prior,
                                                                                       top_k_regulator,
                                                                                       genes_list_u, l_u, r_u,
                                                                                       cell_types_dict)
    total_regulators = regulator_list_neighbor + regulator_list_self

    log_response = True  # take log transformation of the response genes
    Y_train, Y_train_true, Y_test, Y_test_true, response_list = prepare_responses(data_type, datasets_train,
                                                                                  datasets_test,
                                                                                  idx_train_in_general,
                                                                                  idx_test_in_general,
                                                                                  idx_train_in_dataset,
                                                                                  idx_test_in_dataset,
                                                                                  neighbors_train,
                                                                                  neighbors_test,
                                                                                  response_type, log_response,
                                                                                  response_list_prior,
                                                                                  top_k_response,
                                                                                  genes_list_u, l_u, r_u)
    if grid_search:
        X_trains_gs = copy.deepcopy(X_trains)
        Y_train_gs = copy.copy(Y_train)

    # transform features
    transform_features(X_trains, X_tests, feature_types)
    print(f"Minimum value after transformation can below 0: {np.min(X_trains['regulators_self'])}")

    # combine different type of features
    if data_type == 'merfish':
        num_coordinates = 3
    elif data_type == 'starmap' or data_type == 'merfish_cell_line':
        num_coordinates = 2
    else:
        num_coordinates = None

    if np.ndim(X_trains['baseline']) > 1 and np.ndim(X_tests['baseline']) > 1:
        X_train, X_train_clf_1, X_train_clf_2 = combine_features(X_trains, preprocess, num_coordinates)
        X_test, X_test_clf_1, X_test_clf_2 = combine_features(X_tests, preprocess, num_coordinates)
    elif np.ndim(X_trains['baseline']) > 1:
        X_train, X_train_clf_1, X_train_clf_2 = combine_features(X_trains, preprocess, num_coordinates)

    print(f"Dimension of X train is: {X_train.shape}")
    if mode == 'CV':
        print(f"Dimension of X test is: {X_test.shape}")

    # ------ modeling by MESSI ------
    for _i in range(0, n_replicates):

        # ------ set parameters ------
        model_name_gates = 'logistic'
        model_name_experts = 'mrots'

        soft_weights = True
        partial_fit_expert = True

        # if current_cell_type not in ['OD Mature 2', 'Astrocyte', 'Endothelial 1']:
        #     # soft weights
        #     soft_weights = True
        #     partial_fit_expert = True
        #
        # else:
        #     # hard weights
        #     soft_weights = False
        #     partial_fit_expert = False

        # specify default parameters for MESSI
        model_params = {'n_classes_0': n_classes_0,
                        'n_classes_1': n_classes_1,
                        'model_name_gates': model_name_gates,
                        'model_name_experts': model_name_experts,
                        'num_responses': Y_train.shape[1],
                        'soft_weights': soft_weights,
                        'partial_fit_expert': partial_fit_expert,
                        'n_epochs': n_epochs,
                        'tolerance': 3}

        print(f"Model parameters for training is {model_params}")

        # set up directory for saving the model
        sub_condition = f"{condition}_{model_name_gates}_{model_name_experts}"
        sub_dir = f"{data_type}/{behavior_no_space}/{sex}/{current_cell_type_no_space}/{preprocess}/{sub_condition}"
        current_dir = os.path.join(output_path, sub_dir)

        if not os.path.exists(current_dir):
            os.makedirs(current_dir)

        print(f"Model and validation results (if applicable) saved to: {current_dir}")

        if mode == 'CV':
            suffix = f"_{test_animal}_{_i}"
        else:
            suffix = f"_{_i}"

        if grid_search:
            # prepare input meta data
            if data_type == 'merfish':
                meta_per_part = [tuple(i) for i in meta_per_dataset_train]
                meta_idx = meta2idx(idx_train_in_dataset, meta_per_part)
            else:
                meta_per_part, meta_idx = combineParts(samples_train, datasets_train, idx_train_in_dataset)

            # prepare parameters list to be tuned
            if data_type == 'merfish_cell_line':
                current_cell_type_data = 'U-2_OS'
            elif data_type == 'starmap':
                current_cell_type_data = 'STARmap_excitatory'
            else:
                current_cell_type_data = current_cell_type

            params = {'n_classes_1': list(search_range_dict[current_cell_type_data]), 'soft_weights': [True, False],
                      'partial_fit_expert': [True, False]}

            keys, values = zip(*params.items())
            params_list = [dict(zip(keys, v)) for v in itertools.product(*values)]

            new_params_list = []
            for d in params_list:
                if d['soft_weights'] == d['partial_fit_expert']:
                    new_params_list.append(d)
            ratio = 0.2

            # initialize with default values
            model_params_val = model_params.copy()
            model_params_val['n_epochs'] = 1
            model_params_val['tolerance'] = 0
            print(f"Default model parameters for validation {model_params_val}")
            model = hme(**model_params_val)

            gs = gridSearch(params, model, ratio, n_sets, new_params_list)
            gs.generate_val_sets(samples_train, meta_per_part)
            gs.runCV(X_trains_gs, Y_train_gs, meta_per_part, meta_idx, feature_types, data_type,
                     preprocess)
            gs.get_best_parameter()
            print(f"Best params from grid search: {gs.best_params}")

            # modify the parameter setting
            for key, value in gs.best_params.items():
                model_params[key] = value

            print(f"Model parameters for training after grid search {model_params}")

            filename = f"validation_results{suffix}.pickle"
            pickle.dump(gs, open(os.path.join(current_dir, filename), 'wb'))

        # ------ initialize the sample assignments ------

        if grid_search and 'n_classes_1' in params:
            model = AgglomerativeClustering(n_clusters=gs.best_params['n_classes_1'])
        else:
            model = AgglomerativeClustering(n_classes_1)

        model = model.fit(Y_train)
        hier_labels = [model.labels_]
        model_params['init_labels_1'] = hier_labels

        # ------ construct MESSI  ------
        model = hme(**model_params)

        # train
        model.train(X_train, X_train_clf_1, X_train_clf_2, Y_train)

        # save the model
        filename = f"hme_model{suffix}.pickle"
        pickle.dump(model, open(os.path.join(current_dir, filename), 'wb'))

        # predict the left-out animal
        if mode == 'CV':

            Y_hat_final = model.predict(X_test, X_test_clf_1, X_test_clf_2)

            mae = abs(Y_test - Y_hat_final).mean(axis=1).mean()
            print(f"Mean absolute value for {test_animal} is {mae}")

            filename = f"test_predictions_{test_animal}_{_i}"
            np.save(os.path.join(current_dir, filename), Y_hat_final)
            
    print("\n")

Test set is [1]
Training set is [2 3 4]
Preprocess for Excitatory of merfish
19855
11757
Reading file: merfish_animal2_bregma-029.csv
The dimensions of the sample is: (6091, 170)
Reading file: merfish_animal2_bregma-024.csv
The dimensions of the sample is: (6263, 170)
Reading file: merfish_animal2_bregma-019.csv
The dimensions of the sample is: (6328, 170)
Reading file: merfish_animal2_bregma-014.csv
The dimensions of the sample is: (6135, 170)
Reading file: merfish_animal2_bregma-009.csv
The dimensions of the sample is: (5819, 170)
Reading file: merfish_animal2_bregma-004.csv
The dimensions of the sample is: (5693, 170)
Reading file: merfish_animal2_bregma001.csv
The dimensions of the sample is: (5677, 170)
Reading file: merfish_animal2_bregma006.csv
The dimensions of the sample is: (5381, 170)
Reading file: merfish_animal2_bregma011.csv
The dimensions of the sample is: (5727, 170)
Reading file: merfish_animal2_bregma016.csv
The dimensions of the sample is: (5284, 170)
Reading file: m

/home/dora/Documents/research/MESSI/tutorials/messi/hme.py:824: UserWarning: experts weights not sum to 1 for some samples!
  warnings.warn(f"experts weights not sum to 1 for some samples!")


------ epoch 1 ------
Best score: 1000000000.0
Current score: 398113.96558739466
level 1 gate error: 0
level 2 gate error: [0.889138701791625]
experts error: [68364.70856602101, 28370.010229524174, 84819.25195742327, 92086.04970744454, 52496.39037529202, 28540.357866236576, 29015.12230263258, 14421.185444118602]


/home/dora/Documents/research/MESSI/tutorials/messi/hme.py:824: UserWarning: experts weights not sum to 1 for some samples!
  warnings.warn(f"experts weights not sum to 1 for some samples!")


------ epoch 2 ------
Best score: 398113.96558739466
Current score: 387696.0987812748
level 1 gate error: 0
level 2 gate error: [0.7973014146111466]
experts error: [69071.93423886992, 28303.259058824362, 76035.3743100921, 89122.59760322857, 51907.68451985893, 28190.58895373807, 29840.434112959378, 15223.428682288917]


...

------ epoch 6 ------
Best score: 511679.7734559846
Current score: 511908.7536563823
level 1 gate error: 0
level 2 gate error: [0.8746814315009017]
experts error: [73051.25061292296, 66542.12943672585, 94093.87643831733, 35590.054968872544, 46490.620840499316, 26931.537501472023, 31584.2443533004, 28896.269877191356, 87656.79105258304, 21071.103893066007]


/home/dora/Documents/research/MESSI/tutorials/messi/hme.py:824: UserWarning: experts weights not sum to 1 for some samples!
  warnings.warn(f"experts weights not sum to 1 for some samples!")


------ epoch 7 ------
Best score: 511679.7734559846
Current score: 511938.2739003165
level 1 gate error: 0
level 2 gate error: [0.8780618015262682]
experts error: [71503.30865704858, 66831.08125330489, 94877.79702582685, 35697.351658003565, 46733.54783151238, 27754.272986213942, 31506.715569207656, 29056.18084796001, 86957.82385455356, 21019.316154883487]


/home/dora/Documents/research/MESSI/tutorials/messi/hme.py:824: UserWarning: experts weights not sum to 1 for some samples!
  warnings.warn(f"experts weights not sum to 1 for some samples!")


------ epoch 8 ------
Best score: 511679.7734559846
Current score: 512075.5941493139
level 1 gate error: 0
level 2 gate error: [0.8803953398476155]
experts error: [69983.79194259127, 67271.92002338613, 95642.53633135122, 35751.81016138761, 46770.155348418775, 28438.158252652203, 31441.78815789247, 29276.797027156998, 86546.45688604312, 20951.299623094244]
8 epochs in total
Mean absolute value for 4 is 0.37518774373610836




In [8]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 5313.278757810593 seconds ---
